In [1]:
import sys  ###resetmcl1
sys.path.append('../scripts/')
from mcl import *

In [2]:
class ResetMcl(Mcl): 
    def __init__(self, envmap, init_pose, num, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}, \
                 distance_dev_rate=0.14, direction_dev=0.05):
        super().__init__(envmap, init_pose, num, motion_noise_stds, distance_dev_rate, direction_dev)
        self.alphas = {}  #α値の記録（ランドマークの観測数ごと）
        
    def observation_update(self, observation):
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
        
        #alpha値の記録
        alpha = sum([p.weight for p in self.particles])
        obsnum = len(observation)
        if not obsnum in self.alphas: self.alphas[obsnum] = []
        self.alphas[obsnum].append(alpha)
            
        self.set_ml()
        self.resampling() #ここで重みの合計は1になる

In [3]:
def trial():
    time_interval = 0.1
    world = World(300, time_interval, debug=True) 

    ### 地図を生成して3つランドマークを追加 ###
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ### ロボットを作る ###
    initial_pose = np.array([0, 0, 0]).T
    pf = ResetMcl(m, initial_pose, 100)
    circling = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
    r = Robot(initial_pose, sensor=Camera(m), agent=circling, color="red")
    world.append(r)

    world.draw()
    
    return pf

pf = trial()

In [4]:
for num in pf.alphas: ###mclalpharesult
    print("landmarks:", num, "particles:", len(pf.particles), "min:", min(pf.alphas[num]), "max:", max(pf.alphas[num]))

landmarks: 2 particles: 100 min: 1.857484755523877e-08 max: 13.51911651537655
landmarks: 1 particles: 100 min: 2.7996831031680425e-13 max: 18.207671275410252
landmarks: 0 particles: 100 min: 1.0000000000000007 max: 1.0000000000000007


In [5]:
def trial():
    time_interval = 0.1
    world = World(40, time_interval, debug=True) 

    ### 地図を生成して3つランドマークを追加 ###
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ### ロボットを作る ###
    pf = ResetMcl(m, np.array([-4, -4, 0]).T, 100)
    circling = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
    r = Robot(np.array([0,0,0]).T, sensor=Camera(m), agent=circling, color="red")
    world.append(r)

    world.draw()
    
    return pf
    
pf = trial()

In [6]:
for num in pf.alphas: ###mclalpharesult2
    print("landmarks:", num, "particles:", len(pf.particles), "min:", min(pf.alphas[num]), "max:", max(pf.alphas[num]))

landmarks: 2 particles: 100 min: 1.8310874968789142e-121 max: 4.381003519732624e-106
landmarks: 1 particles: 100 min: 0.0 max: 4.114169260003919
landmarks: 0 particles: 100 min: 1.0000000000000007 max: 1.0000000000000007


In [7]:
pf.alphas

{2: [1.8310874968789142e-121, 4.381003519732624e-106, 5.222973459459582e-112],
 1: [4.93537133724464e-111,
  0.0049016942473734644,
  0.0008458380099170069,
  0.006399084258850819,
  0.007611246904031142,
  0.0007694395083310201,
  0.005669981292896277,
  0.008311223139979165,
  0.002160857366996636,
  0.0005613705918674952,
  0.006489440022685712,
  0.0007092994400646107,
  0.000519951248782982,
  0.001168807588396654,
  0.0019397507198203403,
  0.0008176641139069954,
  0.00026658176254945403,
  0.005564131959882897,
  0.0034722626151668184,
  0.00390074204399958,
  0.0011326193748212934,
  4.6167562572315244e-05,
  9.276160574258474e-05,
  5.3054267148524294e-05,
  0.00027195980094126476,
  0.001972940521985085,
  0.004862573102553187,
  0.0007682062428717327,
  0.001216086908405951,
  0.0006615181282953422,
  0.0027022537434811416,
  0.004065706491366195,
  0.001172800430283989,
  0.0033001753025119276,
  0.009176833209964196,
  0.0013029958740195789,
  0.003208191256546746,
  0.001